# Part 1

As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:
```
.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
```
Here, `?` represents text in a language you don't understand. This ticket might be represented as `101,102,103,104,301,302,303,401,402,403`; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:
```
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50
```

your ticket:
`7,1,14`

nearby tickets:
```
7,3,47
40,4,50
55,2,20
38,6,12
```
It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values `4`, `55`, and `12` are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: `4 + 55 + 12 = 71`.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

In [1]:
from collections import namedtuple
from types import SimpleNamespace


class Rule:
    def __init__(self, line: str):
        field, range_rule = line.split(': ')
        ranges = []
        for r in range_rule.split(' or '):
            start, end = [int(i) for i in r.split('-')]
            ranges.append([start, end])
        self.field = field
        self._ranges = ranges
        self._line = line
    
    def is_valid(self, num):
        for r in self._ranges:
            if r[0] <= num and num <= r[1]:
                return True
        return False
    
    def __repr__(self):
        return 'Rule("%s")' % self._line


# Tests
rule = Rule('class: 1-3 or 5-7')
assert rule.field == 'class'
assert rule.is_valid(3) is True
assert rule.is_valid(4) is False
assert rule.is_valid(5) is True


def parse_ticket(line: str):
    return [int(i) for i in line.split(',')]


class InputParser():
    # Just to guard against typos.
    _MODES = SimpleNamespace(**{
        'rules': 'rules', 'your_ticket': 'your ticket', 'nearby_tickets': 'nearby tickets'})
    def __init__(self):
        self._mode = self._MODES.rules
        self.rules = []
        self.your_ticket = None
        self.nearby_tickets = []
    
    def parse_line(self, line: str):
        line = line.rstrip()
        if line == '':
            self._mode = None
        elif line == 'your ticket:':
            self._mode = self._MODES.your_ticket
        elif line == 'nearby tickets:':
            self._mode = self._MODES.nearby_tickets
        else:
            self._parse_line(line)
    
    def _parse_line(self, line):
        if self._mode == self._MODES.rules:
            self.rules.append(Rule(line))
        elif self._mode == self._MODES.your_ticket:
            self.your_ticket = parse_ticket(line)
        elif self._mode == self._MODES.nearby_tickets:
            ticket = parse_ticket(line)
            self.nearby_tickets.append(ticket)
        else:
            raise Exception('Unknown mode.')
            
    def result(self):
        return {'rules': self.rules, 'your_ticket': self.your_ticket, 'nearby_tickets': self.nearby_tickets}

    
def get_input(parser=None):
    if parser is None:
        parser = InputParser()
    with open('inputs/16') as f:
        for line in f.readlines():
             parser.parse_line(line)
    return parser.result()


def result1():
    inp = get_input()
    invalid_values = []
    for ticket in inp['nearby_tickets']:
        for val in ticket:
            if all(not rule.is_valid(val) for rule in inp['rules']):
                invalid_values.append(val)
    return sum(invalid_values)


result1()

27870

# Part 2

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:
```
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19
```
your ticket:
```
11,12,13
```

nearby tickets:
```
3,9,18
15,1,5
5,14,9
```
Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [2]:
from functools import reduce
from operator import mul


def is_valid_ticket(ticket, rules):
    for val in ticket:
        if not any(rule.is_valid(val) for rule in rules):
            return False
    return True


def get_possible_fields_for_values(values, rules):
    return {r.field for r in rules if all(r.is_valid(val) for val in values)}

# Test
test_rules = [Rule('class: 0-1 or 4-19'), Rule('row: 0-5 or 8-19'), Rule('seat: 0-13 or 16-19')]
assert get_possible_fields_for_values([3, 15, 5], test_rules) == {'row'}
assert get_possible_fields_for_values([9, 1, 15], test_rules) == {'row', 'class'}
assert get_possible_fields_for_values([18, 5, 9], test_rules) == {'row', 'class', 'seat'}


def match_columns_with_fieds(tickets, rules):
    unmatched_cols = {}
    for col in range(len(tickets[0])):
        values = [t[col] for t in tickets]
        unmatched_cols[col] = get_possible_fields_for_values(values, rules)
    matches = {}
    # Let's hope that it works with this simple algorithm and there is no need to implement a binary graph
    # matching algorithm
    while len(unmatched_cols) > 0:
        new_matched_cols = {col: list(fields)[0] for col, fields in unmatched_cols.items() if len(fields) == 1}
        if len(new_matched_cols) == 0:
            raise Exception('Unfortunately, the simple method didn\'t work.')
        new_matched_fields = new_matched_cols.values()
        matches.update(new_matched_cols)
        unmatched_cols = {col: fields.difference(new_matched_fields) for col, fields in unmatched_cols.items()
                          if col not in new_matched_cols}
    return matches


# Test
test_tickets = [[3, 9, 18], [15, 1, 5], [5, 14, 9]]
assert match_columns_with_fieds(test_tickets, test_rules) == {0: 'row', 1: 'class', 2: 'seat'}
    

def result2():
    inp = get_input()
    rules = inp['rules']
    your_ticket = inp['your_ticket']
    nearby_tickets = inp['nearby_tickets']
    valid_tickets = [your_ticket] + [t for t in nearby_tickets if is_valid_ticket(t, rules)]
    cols_for_fields = match_columns_with_fieds(valid_tickets, rules)
    departure_cols = [col for col, field in cols_for_fields.items() if field.startswith('departure')]
    return reduce(mul, [your_ticket[col] for col in departure_cols])


result2()

3173135507987